# Lecture 1023

More pandas and charts with Altair using Berkeley 311 call data.

## Import modules
As usual, we'll import modules at the top of the notebook. This time, we don't need the `requests` module since we're not going to download the data from the Internet.

### What is Altair?

[Altair](https://altair-viz.github.io/) is a data visualization library for Python. `matplotlib` is usually the first data viz module Python programmers learn, but Altair is easier to use. The Altair community uses the alias `alt` when importing.

In [1]:
import pandas as pd
import altair as alt

## Import data

We did a lot of work last week cleaning up the Berkeley 311 calls. We don't need to redo that work since we exported a clean version called `berkeley_311_clean.csv`. 

Remember that a `csv` file is just a plain-text file. That means that the file, just as it is, cannot retain the **dtype** of a column.

So this time when we import the data, we'll want to make sure that we set up the dtypes we do know and parse `datetime` dtypes correctly.

I also want to set **Case_ID** to an `object` dtype instead of an `int` dtype. Why would I want to do this? You can't operate on **Case_ID** like it's a number. You aren't going to add up the Case_IDs. So it's better to import that column as an `object`.

In [2]:
berkeley_311 = pd.read_csv('berkeley_311_clean.csv', 
    dtype={
        'Case_ID': object,
    },
    parse_dates=['Date_Opened', 'Date_Closed', 'Close_Time']
)

In [3]:
berkeley_311.head()

,Case_ID,Date_Opened,Case_Status,Date_Closed,Request_Category,Request_SubCategory,Request_Detail,Object_Type,APN,Street_Address,City,State,Neighborhood,Latitude,Longitude,Location,Close_Time
0,121000877593,2021-09-16 06:23:23,Closed,2021-09-20 11:22:22,"Facilities, Electrical & Property Management",Parks/Marina Building Services,Keys / Locks,Property,NaN,"Intersection of Browning and Addison, BERKELEY...",Berkeley,CA,Berkeley,NaN,NaN,NaN,4 days 04:58:59
1,121000876647,2021-09-13 10:50:00,Open,NaT,Refuse and Recycling,Residential,Residential Bulky Pickup,Property,054 180702800,1722 DWIGHT WAY,Berkeley,CA,Berkeley,37.862656,-122.275461,"(37.86265624, -122.27546088)",NaN
2,121000809740,2020-11-06 16:51:00,Closed,2020-11-09 01:52:57,General Questions/information,Miscellaneous,Miscellaneous Service Request,Individual,NaN,NaN,Berkeley,CA,Berkeley,NaN,NaN,NaN,2 days 09:01:57
3,121000809739,2020-11-06 16:38:00,Closed,2020-11-09 01:41:12,General Questions/information,Miscellaneous,Miscellaneous Service Request,Property,060 249305600,1411 GRIZZLY PEAK BLVD,Berkeley,CA,Berkeley,37.884799,-122.247874,"(37.88479918, -122.24787412)",2 days 09:03:12
4,121000793663,2020-09-01 11:32:00,Closed,2020-09-01 11:36:00,Other Account Services and Billing,Marina,Payment Collection - Marina,Individual,NaN,NaN,Berkeley,CA,Berkeley,NaN,NaN,NaN,0 days 00:04:00


In [4]:
berkeley_311.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648195 entries, 0 to 648194
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Case_ID              648195 non-null  object        
 1   Date_Opened          648195 non-null  datetime64[ns]
 2   Case_Status          648195 non-null  object        
 3   Date_Closed          615523 non-null  datetime64[ns]
 4   Request_Category     648195 non-null  object        
 5   Request_SubCategory  648195 non-null  object        
 6   Request_Detail       648195 non-null  object        
 7   Object_Type          648195 non-null  object        
 8   APN                  371225 non-null  object        
 9   Street_Address       408719 non-null  object        
 10  City                 648195 non-null  object        
 11  State                648195 non-null  object        
 12  Neighborhood         648195 non-null  object        
 13  Latitude      

The **Close_Time** column didn't get typed as `timedelta`. It doesn't look like it's possible to do so with `pd.read_csv()`. So we'll just set it this way:

In [5]:
berkeley_311['Close_Time'] = pd.to_timedelta(berkeley_311['Close_Time']) 

In [6]:
berkeley_311.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648195 entries, 0 to 648194
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype          
---  ------               --------------   -----          
 0   Case_ID              648195 non-null  object         
 1   Date_Opened          648195 non-null  datetime64[ns] 
 2   Case_Status          648195 non-null  object         
 3   Date_Closed          615523 non-null  datetime64[ns] 
 4   Request_Category     648195 non-null  object         
 5   Request_SubCategory  648195 non-null  object         
 6   Request_Detail       648195 non-null  object         
 7   Object_Type          648195 non-null  object         
 8   APN                  371225 non-null  object         
 9   Street_Address       408719 non-null  object         
 10  City                 648195 non-null  object         
 11  State                648195 non-null  object         
 12  Neighborhood         648195 non-null  object         
 13 

In [7]:
berkeley_311

,Case_ID,Date_Opened,Case_Status,Date_Closed,Request_Category,Request_SubCategory,Request_Detail,Object_Type,APN,Street_Address,City,State,Neighborhood,Latitude,Longitude,Location,Close_Time
0,121000877593,2021-09-16 06:23:23,Closed,2021-09-20 11:22:22,"Facilities, Electrical & Property Management",Parks/Marina Building Services,Keys / Locks,Property,NaN,"Intersection of Browning and Addison, BERKELEY...",Berkeley,CA,Berkeley,NaN,NaN,NaN,4 days 04:58:59
1,121000876647,2021-09-13 10:50:00,Open,NaT,Refuse and Recycling,Residential,Residential Bulky Pickup,Property,054 180702800,1722 DWIGHT WAY,Berkeley,CA,Berkeley,37.862656,-122.275461,"(37.86265624, -122.27546088)",NaT
2,121000809740,2020-11-06 16:51:00,Closed,2020-11-09 01:52:57,General Questions/information,Miscellaneous,Miscellaneous Service Request,Individual,NaN,NaN,Berkeley,CA,Berkeley,NaN,NaN,NaN,2 days 09:01:57
3,121000809739,2020-11-06 16:38:00,Closed,2020-11-09 01:41:12,General Questions/information,Miscellaneous,Miscellaneous Service Request,Property,060 249305600,1411 GRIZZLY PEAK BLVD,Berkeley,CA,Berkeley,37.884799,-122.247874,"(37.88479918, -122.24787412)",2 days 09:03:12
4,121000793663,2020-09-01 11:32:00,Closed,2020-09-01 11:36:00,Other Account Services and Billing,Marina,Payment Collection - Marina,Individual,NaN,NaN,Berkeley,CA,Berkeley,NaN,NaN,NaN,0 days 00:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648190,121000963461,2022-10-22 12:00:20,Open,NaT,General Questions/information,Miscellaneous,Miscellaneous Internet Request,Property,059 226603200,1413 BONITA AVE,Berkeley,CA,Berkeley,37.881127,-122.272577,"(37.8811269, -122.2725769)",NaT
648191,121000963479,2022-10-22 22:52:40,Open,NaT,General Questions/information,Miscellaneous,Miscellaneous Internet Request,Individual,NaN,NaN,Berkeley,CA,Berkeley,NaN,NaN,NaN,NaT
648192,121000963164,2022-10-20 19:50:00,Closed,2022-10-22 07:12:00,General Questions/information,Miscellaneous,Miscellaneous Internet Request,Property,054 173001100,1617 STUART ST,Berkeley,CA,Berkeley,37.857543,-122.276810,"(37.85754346, -122.27680958)",1 days 11:22:00
648193,121000963464,2022-10-22 13:51:47,Open,NaT,General Questions/information,Miscellaneous,Miscellaneous Internet Request,Individual,NaN,NaN,Berkeley,CA,Berkeley,NaN,NaN,NaN,NaT


## Explore data

What do I do if I don't have a question yet? I'm not really sure what to look into with this 311 data. So I'm going to explore it a little bit. I might do some analysis, I might not.

### Categories of incidents in 2019

I'm curious about the different categories of incidents in the year 2019.

First, I'll create a new dataframe `berkeley_311_2019` that limits the `berkeley_311` data to just the cases that were open in 2019.

In [8]:
berkeley_311_2019 = berkeley_311[
    (berkeley_311['Date_Opened'] >= '2019-01-01') &
    (berkeley_311['Date_Opened'] < '2020-01-01') # Why don't I use `berkeley_311['Date_Opened'] <= '2019-12-31']` ?
].copy()

In [9]:
berkeley_311_2019

,Case_ID,Date_Opened,Case_Status,Date_Closed,Request_Category,Request_SubCategory,Request_Detail,Object_Type,APN,Street_Address,City,State,Neighborhood,Latitude,Longitude,Location,Close_Time
47,121000737005,2019-12-11 12:16:00,Closed,2019-12-11 14:22:00,Public Records Act,Request,Police,Organisation,NaN,NaN,Berkeley,CA,Berkeley,NaN,NaN,NaN,0 days 02:06:00
52,121000655799,2019-01-16 15:29:00,Closed,2019-01-16 15:33:00,Public Records Act,Request,Public Works,Unknown,NaN,NaN,Berkeley,CA,Berkeley,NaN,NaN,NaN,0 days 00:04:00
54,121000736811,2019-12-11 08:46:00,Closed,2019-12-11 14:13:00,"Streets, Utilities, and Transportation",Clean City Program,Illegal Dumping - City Property,Property,056 199300800,2222 ACTON ST,Berkeley,CA,Berkeley,37.866180,-122.284156,"(37.86617979, -122.28415637)",0 days 05:27:00
56,121000736608,2019-12-10 11:20:39,Closed,2019-12-11 11:03:07,Refuse and Recycling,Residential,Residential Missed Pickup Integration,Property,053 169600300,2712 STUART ST,Berkeley,CA,Berkeley,37.859712,-122.252652,"(37.85971233, -122.2526523)",0 days 23:42:28
58,121000736794,2019-12-10 16:56:00,Closed,2019-12-11 08:39:25,General Questions/information,Miscellaneous,Miscellaneous Internet Request,Property,062 289002200,651 VINCENTE AVE,Berkeley,CA,Berkeley,37.895913,-122.282826,"(37.89591259, -122.28282581)",0 days 15:43:25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638144,121000685087,2019-05-22 16:34:00,Closed,2019-05-23 12:48:00,"Streets, Utilities, and Transportation",Clean City Program,Illegal Dumping - City Property,Unknown,NaN,NaN,Berkeley,CA,Berkeley,NaN,NaN,NaN,0 days 20:14:00
638145,121000685430,2019-05-23 18:44:00,Closed,2019-05-24 09:50:41,General Questions/information,Miscellaneous,Miscellaneous Internet Request,Individual,NaN,NaN,Berkeley,CA,Berkeley,NaN,NaN,NaN,0 days 15:06:41
638146,121000685503,2019-05-24 10:01:29,Closed,2019-05-24 15:16:51,Other Account Services and Billing,Marina,Payment Collection - Marina,Individual,NaN,NaN,Berkeley,CA,Berkeley,NaN,NaN,NaN,0 days 05:15:22
638147,121000685121,2019-05-23 07:33:00,Closed,2019-05-24 14:12:00,Graffiti and Vandalism,Graffiti,Graffiti Abatement - Traffic Maintenance,Property,NaN,"Intersection of Valley and Channing, BERKELEY, CA",Berkeley,CA,Berkeley,NaN,NaN,NaN,1 days 06:39:00


One thing I'm seeing immediately is that the index of this new dataframe `berkeley_311_2019` looks kind of weird. It's no longer sequential. I can reset the index to make it sequential by using `df.reset_index(drop=True)`.

```python
berkeley_311_2019 = berkeley_311_2019.reset_index(drop=True)
```

Alternatively, instead of copying the original dataframe with df.copy(), we can reset the index at the same time we subset the data:

In [10]:
berkeley_311_2019 = berkeley_311[
    (berkeley_311['Date_Opened'] >= '2019-01-01') &
    (berkeley_311['Date_Opened'] < '2020-01-01')
].reset_index(drop=True)

In [11]:
berkeley_311_2019

,Case_ID,Date_Opened,Case_Status,Date_Closed,Request_Category,Request_SubCategory,Request_Detail,Object_Type,APN,Street_Address,City,State,Neighborhood,Latitude,Longitude,Location,Close_Time
0,121000737005,2019-12-11 12:16:00,Closed,2019-12-11 14:22:00,Public Records Act,Request,Police,Organisation,NaN,NaN,Berkeley,CA,Berkeley,NaN,NaN,NaN,0 days 02:06:00
1,121000655799,2019-01-16 15:29:00,Closed,2019-01-16 15:33:00,Public Records Act,Request,Public Works,Unknown,NaN,NaN,Berkeley,CA,Berkeley,NaN,NaN,NaN,0 days 00:04:00
2,121000736811,2019-12-11 08:46:00,Closed,2019-12-11 14:13:00,"Streets, Utilities, and Transportation",Clean City Program,Illegal Dumping - City Property,Property,056 199300800,2222 ACTON ST,Berkeley,CA,Berkeley,37.866180,-122.284156,"(37.86617979, -122.28415637)",0 days 05:27:00
3,121000736608,2019-12-10 11:20:39,Closed,2019-12-11 11:03:07,Refuse and Recycling,Residential,Residential Missed Pickup Integration,Property,053 169600300,2712 STUART ST,Berkeley,CA,Berkeley,37.859712,-122.252652,"(37.85971233, -122.2526523)",0 days 23:42:28
4,121000736794,2019-12-10 16:56:00,Closed,2019-12-11 08:39:25,General Questions/information,Miscellaneous,Miscellaneous Internet Request,Property,062 289002200,651 VINCENTE AVE,Berkeley,CA,Berkeley,37.895913,-122.282826,"(37.89591259, -122.28282581)",0 days 15:43:25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58628,121000685087,2019-05-22 16:34:00,Closed,2019-05-23 12:48:00,"Streets, Utilities, and Transportation",Clean City Program,Illegal Dumping - City Property,Unknown,NaN,NaN,Berkeley,CA,Berkeley,NaN,NaN,NaN,0 days 20:14:00
58629,121000685430,2019-05-23 18:44:00,Closed,2019-05-24 09:50:41,General Questions/information,Miscellaneous,Miscellaneous Internet Request,Individual,NaN,NaN,Berkeley,CA,Berkeley,NaN,NaN,NaN,0 days 15:06:41
58630,121000685503,2019-05-24 10:01:29,Closed,2019-05-24 15:16:51,Other Account Services and Billing,Marina,Payment Collection - Marina,Individual,NaN,NaN,Berkeley,CA,Berkeley,NaN,NaN,NaN,0 days 05:15:22
58631,121000685121,2019-05-23 07:33:00,Closed,2019-05-24 14:12:00,Graffiti and Vandalism,Graffiti,Graffiti Abatement - Traffic Maintenance,Property,NaN,"Intersection of Valley and Channing, BERKELEY, CA",Berkeley,CA,Berkeley,NaN,NaN,NaN,1 days 06:39:00


#### Let's see all the unique values of **Request_Category**

You can call `series.unique()` on a column:

In [12]:
berkeley_311_2019['Request_Category'].unique()

array(['Public Records Act', 'Streets, Utilities, and Transportation',
       'Refuse and Recycling', 'General Questions/information',
       'Other Account Services and Billing', 'Traffic and Transportation',
       'Facilities, Electrical & Property Management',
       'Government Activity', 'Parks, Trees and Vegetation',
       'Graffiti and Vandalism', 'Equipment Maintenance',
       'Disability Compliance', 'Business License', 'Outside Agencies',
       'Environmental Services and Programs'], dtype=object)

I'm interested in getting a count of categories for 2019. How can I achieve this?

#### Aggregate with `df.groupby()`

To aggregate the data, we're going to use a method called `df.groupby()`. Normally, when we group data, we'll group them by columns, like so:

```python
df.groupby(['Column 1', 'Column 2'])
```

You can also just group by a single column, like we're doing below:

In [13]:
berkeley_311_2019.groupby(['Request_Category'])

Running a `df.groupby()` doesn't do anything on its own, it just creates a pandas DataFrameGroupBy object. You have to follow it up with some kind of other method. Below, we're calling `df.count()` on the DataFrameGroupBy object.

In [14]:
berkeley_311_2019.groupby(['Request_Category']).count()

,Case_ID,Date_Opened,Case_Status,Date_Closed,Request_SubCategory,Request_Detail,Object_Type,APN,Street_Address,City,State,Neighborhood,Latitude,Longitude,Location,Close_Time
Request_Category,,,,,,,,,,,,,,,,
Business License,3,3,3,3,3,3,3,0,0,3,3,3,0,0,0,3
Disability Compliance,8,8,8,8,8,8,8,0,0,8,8,8,0,0,0,8
Environmental Services and Programs,1,1,1,1,1,1,1,0,0,1,1,1,0,0,0,1
Equipment Maintenance,43,43,43,41,43,43,43,19,35,43,43,43,19,19,19,41
"Facilities, Electrical & Property Management",1041,1041,1041,995,1041,1041,1041,488,742,1041,1041,1041,487,487,487,995
General Questions/information,9561,9561,9561,9292,9561,9561,9561,5782,6386,9561,9561,9561,5712,5712,5712,9292
Government Activity,1153,1153,1153,1153,1153,1153,1153,547,1038,1153,1153,1153,543,543,543,1153
Graffiti and Vandalism,419,419,419,395,419,419,419,266,386,419,419,419,265,265,265,395
Other Account Services and Billing,5385,5385,5385,5381,5385,5385,5385,167,175,5385,5385,5385,161,161,161,5381


OK! So that's a new dataframe, with a little too much info. The "counts" are also different across columns. For "count" to make sense, you need to limit the dataframe to a column with zero null cells. Since we know **Case_ID** has unique values, we just need that one. 

Let's also set the results of the groupby/count to a new dataframe called `category_counts_2019`. You won't need to use `df.copy()` here because pandas will create a new dataframe instance with `df.groupby().count()`.

In [15]:
category_counts_2019 = berkeley_311_2019.groupby(['Request_Category']).count()

Now let's subset just the one column, **Case_ID**, from `category_counts_2019`:

In [16]:
category_counts_2019 = category_counts_2019[['Case_ID']].copy()
category_counts_2019

,Case_ID
Request_Category,
Business License,3
Disability Compliance,8
Environmental Services and Programs,1
Equipment Maintenance,43
"Facilities, Electrical & Property Management",1041
General Questions/information,9561
Government Activity,1153
Graffiti and Vandalism,419
Other Account Services and Billing,5385


Let's look at that code one more time.
```python
category_counts_2019 = category_counts_2019[['Case_ID']].copy()
```

The outer set of brackets is used to select the data. Within the outer set of brackets is `['Case_ID']`. That is a list of the columns. If we wanted to select two columns, we might have written:

```python
category_counts_2019 = category_counts_2019[['Case_ID', 'Date_Opened']].copy()
```

#### Resetting the index

In this dataframe, the index is no longer a series of sequential integers like we've seen before. We want **Request_Category** to be a column, not an index, because that makes the dataframe easier to use.

We're going to use `df.reset_index()`. This time, we're not going to use the `drop=True` argument because we want to create a wholly new index.

In [17]:
category_counts_2019 = category_counts_2019.reset_index()

In [18]:
category_counts_2019

,Request_Category,Case_ID
0,Business License,3
1,Disability Compliance,8
2,Environmental Services and Programs,1
3,Equipment Maintenance,43
4,"Facilities, Electrical & Property Management",1041
5,General Questions/information,9561
6,Government Activity,1153
7,Graffiti and Vandalism,419
8,Other Account Services and Billing,5385
9,Outside Agencies,1


#### Rename columns

Let's change the column names, while we're at it. 
I want to change the name of the **Case_ID** column to represent an actual count, like **Case_Count**.

You can replace _all_ the columns in a dataframe with the following code:

```python
category_counts_2019.columns = ['Request_Category', 'Case_Count']
```

If you have a lot of columns, that's going to be a long list. But if you have a lot of columns to rename, the method above might be easier. If you have only one column to rename out of many columns, you'll want to use the following code:

```python
category_counts_2019.rename(columns={'Case_ID': 'Case_Count'}, inplace=True)
```

The first argument in the `df.rename()` method is `columns`. And what do we set columns to? We set it to a Python dictionary where the "key" is the original column name and the "value" is the new column name: `{'Case_ID': 'Case_Count'}`. 

The second argument is `inplace=True`. That tells us to change the `category_counts_2019` "in place" or without having to reset the dataframe variable. A lot of the methods in pandas return a new dataframe instead of altering the original dataframe. An alternative to using `inplace` is the following code:

```python
category_counts_2019 = category_counts_2019.rename(columns={'Case_ID': 'Count'})
```

In [19]:
category_counts_2019.rename(columns={'Case_ID': 'Case_Count'}, inplace=True)

Let's take a look at our nicely named summary table:

In [20]:
category_counts_2019

,Request_Category,Case_Count
0,Business License,3
1,Disability Compliance,8
2,Environmental Services and Programs,1
3,Equipment Maintenance,43
4,"Facilities, Electrical & Property Management",1041
5,General Questions/information,9561
6,Government Activity,1153
7,Graffiti and Vandalism,419
8,Other Account Services and Billing,5385
9,Outside Agencies,1


Now, I'll sort the dataframe by **Case_Count**:

In [21]:
category_counts_2019 = category_counts_2019.sort_values(by=['Case_Count'], ascending=False).copy()
category_counts_2019

,Request_Category,Case_Count
12,Refuse and Recycling,28063
5,General Questions/information,9561
13,"Streets, Utilities, and Transportation",7461
8,Other Account Services and Billing,5385
10,"Parks, Trees and Vegetation",3499
6,Government Activity,1153
14,Traffic and Transportation,1051
4,"Facilities, Electrical & Property Management",1041
11,Public Records Act,944
7,Graffiti and Vandalism,419


Looks like `Refuse and Recycling`, along with `General Questions/information` and `Streets, Utilities, and Transportation` were among the top issues in 2019. Might be worth looking into some of the sub-categories later.

At this point, you can see the index is out of order now. You can use `df.reset_index(drop=True)` if you like. You could have also replaced `df.copy()` in the previous cell with `df.reset_index(drop=True)`.

In [22]:
category_counts_2019 = category_counts_2019.reset_index(drop=True)
category_counts_2019

,Request_Category,Case_Count
0,Refuse and Recycling,28063
1,General Questions/information,9561
2,"Streets, Utilities, and Transportation",7461
3,Other Account Services and Billing,5385
4,"Parks, Trees and Vegetation",3499
5,Government Activity,1153
6,Traffic and Transportation,1051
7,"Facilities, Electrical & Property Management",1041
8,Public Records Act,944
9,Graffiti and Vandalism,419


#### Let's visualize this summary table!

Before we run the Altair code below, let's take a closer look:

```python
alt.Chart(category_counts_2019).mark_bar().encode(
    x='Case_Count',
    y='Request_Category'
).properties(
    title='Berkeley 311 cases in 2019'
)
```
The first part of the code `alt.Chart()` requires you to fill the first argument with a dataframe, in this case `category_counts_2019`.

The next part of the code `mark_bar()` specifies a bar chart. (If you want a line chart, you'd use `mark_line()`.)

After that, `.encode()` tells Altair which columns to use for the `x` and `y` axes.

If you want to add a title, you'd use Altair's `.properties()` method.

In [23]:
alt.Chart(category_counts_2019).mark_bar().encode(
    x='Case_Count',
    y='Request_Category'
).properties(
    title='Berkeley 311 cases in 2019'
)

/Users/soo/.pyenv/versions/3.9.4/envs/j233-files-3.9.4/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

Annoyingly, this doesn't sort the chart in descending order, which I prefer. This is the code to do, it's a little more complicated:

```python
alt.Chart(category_counts_2019).mark_bar().encode(
    x='Case_Count',
    y=alt.Y('Request_Category', sort='-x')
).properties(
    title='Berkeley 311 cases in 2019'
)
```

Basically, you have to create a custom Y encoding with the format: `alt.Y('column_name', sort='-x')`. `-x` means the inverse of the x-axis, in this case. This is not intuitive, I think — it's just something you'd have to look up in the documentation.

In [24]:
alt.Chart(category_counts_2019).mark_bar().encode(
    x='Case_Count',
    y=alt.Y('Request_Category', sort='-x')
).properties(
    title='Berkeley 311 cases in 2019'
)

alt.Chart(...)

### Count how many incidents per year

The next thing I'd like to do is get a count of all the incidents by year. However, I know from the last notebook that the data for 2010 and 2022 are not complete. So I need to subset.

Below, I'm creating a new dataframe called `berkeley_311_complete` that limits the `berkeley_311` dataframe to ones in which the **Date_Opened** value starts on or after January 1, 2011 and is before January 1, 2021. 

In [25]:
berkeley_311_complete = berkeley_311[
    (berkeley_311['Date_Opened'] >= '2011-01-01') &
    (berkeley_311['Date_Opened'] < '2021-01-01')
].reset_index(drop=True)

#### Use df.groupby() with datetimes

It's a little tricky to group by datetimes. Instead of grouping by just a column name, we're going to have to use a method called `pd.Grouper`. 

Before we run the code below, let's look at the different arguments within the method:

```python
pd.Grouper(key='Date_Opened', axis=0, freq='A')
```

The `key` argument lists the column. The `axis` argument is `0`. In pandas, axis 0 is rows and axis 1 means columns. That means you can do column-wise calculations if your data is shaped differently. 

The `freq` argument is `A`, which stands for "annual" or year (`Y` also works, but isn't documented). You can see other [frequency arguments](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases) in the official pandas documentation.

In [26]:
berkeley_311_complete.groupby([pd.Grouper(key='Date_Opened', axis=0, freq='A')])

Remember that running a `df.groupby()` doesn't do anything on its own; you have to chain that command with some kind of other method. Below, we're calling `df.count()` on the DataFrameGroupBy object. Finally, we're calling our new dataframe `annual_cases`.

In [27]:
annual_cases = berkeley_311_complete.groupby([pd.Grouper(key='Date_Opened', axis=0, freq='A')]).count()

In [28]:
annual_cases

,Case_ID,Case_Status,Date_Closed,Request_Category,Request_SubCategory,Request_Detail,Object_Type,APN,Street_Address,City,State,Neighborhood,Latitude,Longitude,Location,Close_Time
Date_Opened,,,,,,,,,,,,,,,,
2011-12-31,39708,39708,39703,39708,39708,39708,39708,17405,18215,39708,39708,39708,17266,17266,17266,39703
2012-12-31,46643,46643,46636,46643,46643,46643,46643,20919,22488,46643,46643,46643,20705,20705,20705,46636
2013-12-31,50309,50309,50233,50309,50309,50309,50309,25063,27362,50309,50309,50309,24791,24791,24791,50233
2014-12-31,53531,53531,53238,53531,53531,53531,53531,28423,31356,53531,53531,53531,28148,28148,28148,53238
2015-12-31,50334,50334,50029,50334,50334,50334,50334,27432,30466,50334,50334,50334,27087,27087,27087,50029
2016-12-31,50573,50573,50083,50573,50573,50573,50573,28281,31870,50573,50573,50573,27896,27896,27896,50083
2017-12-31,51325,51325,49419,51325,51325,51325,51325,29776,34188,51325,51325,51325,29395,29395,29395,49419
2018-12-31,57636,57636,54508,57636,57636,57636,57636,37882,42171,57636,57636,57636,37361,37361,37361,54508
2019-12-31,58633,58633,54696,58633,58633,58633,58633,39485,43863,58633,58633,58633,38952,38952,38952,54696


Now let's subset just the one column, **Case_ID**, from annual cases, then reset the index so that `Date_Opened` becomes a new column:

In [29]:
annual_cases = annual_cases[['Case_ID']].reset_index()
annual_cases

,Date_Opened,Case_ID
0,2011-12-31,39708
1,2012-12-31,46643
2,2013-12-31,50309
3,2014-12-31,53531
4,2015-12-31,50334
5,2016-12-31,50573
6,2017-12-31,51325
7,2018-12-31,57636
8,2019-12-31,58633
9,2020-12-31,56145


#### Rename columns

In [30]:
annual_cases.rename(columns={'Case_ID': 'Case_Count'}, inplace=True)

Let's take a look at our nicely named summary table:

In [31]:
annual_cases

,Date_Opened,Case_Count
0,2011-12-31,39708
1,2012-12-31,46643
2,2013-12-31,50309
3,2014-12-31,53531
4,2015-12-31,50334
5,2016-12-31,50573
6,2017-12-31,51325
7,2018-12-31,57636
8,2019-12-31,58633
9,2020-12-31,56145


Let's create a new column in `annual_cases` called **Year**.

In [32]:
annual_cases['Year'] = annual_cases['Date_Opened'].dt.year

In [33]:
annual_cases

,Date_Opened,Case_Count,Year
0,2011-12-31,39708,2011
1,2012-12-31,46643,2012
2,2013-12-31,50309,2013
3,2014-12-31,53531,2014
4,2015-12-31,50334,2015
5,2016-12-31,50573,2016
6,2017-12-31,51325,2017
7,2018-12-31,57636,2018
8,2019-12-31,58633,2019
9,2020-12-31,56145,2020


At this point, I don't need the **Date_Opened** column anymore. So I can subset the dataframe with just the two columns I need. 

In [34]:
annual_cases = annual_cases[['Year', 'Case_Count']].copy()

In [35]:
annual_cases

,Year,Case_Count
0,2011,39708
1,2012,46643
2,2013,50309
3,2014,53531
4,2015,50334
5,2016,50573
6,2017,51325
7,2018,57636
8,2019,58633
9,2020,56145


#### Visualize

In [36]:
alt.Chart(annual_cases).mark_bar().encode(
    x='Year',
    y='Case_Count'
)

/Users/soo/.pyenv/versions/3.9.4/envs/j233-files-3.9.4/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

That's pretty cool, but **Year** shows up kind of weird. Let's make a very small alteration to the code.

Before you run the code below, notice that after `Year` there's a colon and an `O`. The `O` is shorthand for "ordinal," and tells Altair to treat `Year` as if it's a discrete quantity (a.k.a. integers), not a continuous quantity (e.g. a number with decimals). 

In [37]:
alt.Chart(annual_cases).mark_bar().encode(
    x='Year:O',
    y='Case_Count'
).properties(
    title='Berkeley 311 calls: Number of cases'
)

alt.Chart(...)

You can read about more [Altair encoding types](https://altair-viz.github.io/user_guide/encoding.html#encoding-data-types) in the documentation. It's helpful to get familiar with those encoding types in the event your chart doesn't look quite right. Try adjusting the encoding types on your own to see what happens.

### Median Close_Time by year

Now I'd like to try to get the median length of time it takes to close a case by year. I'm going to try something I think will work...

In [38]:
median_close_time = berkeley_311_complete.groupby([pd.Grouper(key='Date_Opened', axis=0, freq='A') ]).median()

/var/folders/21/d8yypr810tq3zs4fg6dw8w1c0000gn/T/ipykernel_12975/442100196.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  median_close_time = berkeley_311_complete.groupby([pd.Grouper(key='Date_Opened', axis=0, freq='A') ]).median()


In [39]:
median_close_time

,Latitude,Longitude
Date_Opened,,
2011-12-31,37.868572,-122.272987
2012-12-31,37.868733,-122.272429
2013-12-31,37.868331,-122.271909
2014-12-31,37.868510,-122.272588
2015-12-31,37.868306,-122.272719
2016-12-31,37.867818,-122.273014
2017-12-31,37.868334,-122.272719
2018-12-31,37.868004,-122.274118
2019-12-31,37.868199,-122.272885


It looks like that didn't work! Sometimes pandas doesn't work the way you want it to. Getting the median of a `timedelta` dtype is a feature that hasn't yet been built.

I'm instead going to create a new column called **Close_Time_Seconds** that converts **Close_Time** to seconds. Then I can calculate the median number of seconds.

In [40]:
berkeley_311_complete['Close_Time_Seconds'] = berkeley_311_complete['Close_Time'].dt.total_seconds()

Let's make sure the new column looks OK:

In [41]:
berkeley_311_complete.head()

,Case_ID,Date_Opened,Case_Status,Date_Closed,Request_Category,Request_SubCategory,Request_Detail,Object_Type,APN,Street_Address,City,State,Neighborhood,Latitude,Longitude,Location,Close_Time,Close_Time_Seconds
0,121000809740,2020-11-06 16:51:00,Closed,2020-11-09 01:52:57,General Questions/information,Miscellaneous,Miscellaneous Service Request,Individual,NaN,NaN,Berkeley,CA,Berkeley,NaN,NaN,NaN,2 days 09:01:57,205317.0
1,121000809739,2020-11-06 16:38:00,Closed,2020-11-09 01:41:12,General Questions/information,Miscellaneous,Miscellaneous Service Request,Property,060 249305600,1411 GRIZZLY PEAK BLVD,Berkeley,CA,Berkeley,37.884799,-122.247874,"(37.88479918, -122.24787412)",2 days 09:03:12,205392.0
2,121000793663,2020-09-01 11:32:00,Closed,2020-09-01 11:36:00,Other Account Services and Billing,Marina,Payment Collection - Marina,Individual,NaN,NaN,Berkeley,CA,Berkeley,NaN,NaN,NaN,0 days 00:04:00,240.0
3,121000797043,2020-09-15 13:07:00,Closed,2020-11-09 12:15:48,General Questions/information,Miscellaneous,Miscellaneous Internet Request,Individual,NaN,NaN,Berkeley,CA,Berkeley,NaN,NaN,NaN,54 days 23:08:48,4748928.0
4,121000793489,2020-08-31 15:53:00,Closed,2021-03-18 22:40:00,Government Activity,Inquiry,Encampment Complaint,Street,NaN,NaN,Berkeley,CA,Berkeley,NaN,NaN,NaN,199 days 06:47:00,17218020.0


Let's run the groupby/median code we used before. Hopefully pandas will calculate **Close_Time_Seconds** this time around:

In [42]:
median_close_time = berkeley_311_complete.groupby([pd.Grouper(key='Date_Opened', axis=0, freq='A'), ]).median()

/var/folders/21/d8yypr810tq3zs4fg6dw8w1c0000gn/T/ipykernel_12975/3109077074.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  median_close_time = berkeley_311_complete.groupby([pd.Grouper(key='Date_Opened', axis=0, freq='A'), ]).median()


In [43]:
median_close_time

,Latitude,Longitude,Close_Time_Seconds
Date_Opened,,,
2011-12-31,37.868572,-122.272987,7449028.0
2012-12-31,37.868733,-122.272429,1382708.0
2013-12-31,37.868331,-122.271909,688226.0
2014-12-31,37.868510,-122.272588,930389.5
2015-12-31,37.868306,-122.272719,374478.0
2016-12-31,37.867818,-122.273014,338033.0
2017-12-31,37.868334,-122.272719,314478.0
2018-12-31,37.868004,-122.274118,274395.5
2019-12-31,37.868199,-122.272885,320116.5


That worked! Now we can subset for the columns we want, and set **Date_Opened** to its own column.

In [44]:
median_close_time = median_close_time[['Close_Time_Seconds']].reset_index()
median_close_time

,Date_Opened,Close_Time_Seconds
0,2011-12-31,7449028.0
1,2012-12-31,1382708.0
2,2013-12-31,688226.0
3,2014-12-31,930389.5
4,2015-12-31,374478.0
5,2016-12-31,338033.0
6,2017-12-31,314478.0
7,2018-12-31,274395.5
8,2019-12-31,320116.5
9,2020-12-31,189712.0


Below, I'm creating a new column called **Year**, as we did before. I'm also going to convert seconds to days:

In [45]:
median_close_time['Year'] = median_close_time['Date_Opened'].dt.year
median_close_time['Close_Time_Days'] = median_close_time['Close_Time_Seconds'] / 60 / 60 / 24

Let's take another look at our dataframe before we subset:

In [46]:
median_close_time

,Date_Opened,Close_Time_Seconds,Year,Close_Time_Days
0,2011-12-31,7449028.0,2011,86.215602
1,2012-12-31,1382708.0,2012,16.003565
2,2013-12-31,688226.0,2013,7.965579
3,2014-12-31,930389.5,2014,10.768397
4,2015-12-31,374478.0,2015,4.334236
5,2016-12-31,338033.0,2016,3.912419
6,2017-12-31,314478.0,2017,3.639792
7,2018-12-31,274395.5,2018,3.175874
8,2019-12-31,320116.5,2019,3.705052
9,2020-12-31,189712.0,2020,2.195741


In [47]:
median_close_time = median_close_time[['Year', 'Close_Time_Days']].copy()

In [48]:
median_close_time

,Year,Close_Time_Days
0,2011,86.215602
1,2012,16.003565
2,2013,7.965579
3,2014,10.768397
4,2015,4.334236
5,2016,3.912419
6,2017,3.639792
7,2018,3.175874
8,2019,3.705052
9,2020,2.195741


In [49]:
alt.Chart(median_close_time[['Year', 'Close_Time_Days']]).mark_bar().encode(
    x='Year:O',
    y='Close_Time_Days',
).properties(
    title='Berkeley 311 calls: Median resolution time'
)

/Users/soo/.pyenv/versions/3.9.4/envs/j233-files-3.9.4/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

### Merge two dataframes

Now I'd like to merge `median_close_time` and `annual_cases`. Why? Mostly because I'd like to teach you how to merge dataframes. But you can get a neat summary table this way. Let's look at both dataframes again:

In [50]:
annual_cases

,Year,Case_Count
0,2011,39708
1,2012,46643
2,2013,50309
3,2014,53531
4,2015,50334
5,2016,50573
6,2017,51325
7,2018,57636
8,2019,58633
9,2020,56145


In [51]:
median_close_time

,Year,Close_Time_Days
0,2011,86.215602
1,2012,16.003565
2,2013,7.965579
3,2014,10.768397
4,2015,4.334236
5,2016,3.912419
6,2017,3.639792
7,2018,3.175874
8,2019,3.705052
9,2020,2.195741


Let's look at the arguments in `pd.merge()` before we run it:

```python
pd.merge(
    df1,
    df2,
    how='outer', # other options: 'inner', 'left', 'right'
    on='Year',
    validate='1:1' # options: '1:m', 'm:m', 'm:1'
)
```
1. The first argument is the left-hand dataframe. The second argument is the right-hand dataframe. Why is it important that there's an order to dataframes? 

2. The `how` argument tells pandas how we'll merge the two dataframes. In this case, we'll use `outer`. But we could also use `left`, `right`, or `inner`. What does this mean? Let's take a look at the lecture slides to view a visual. In this case, it doesn't matter which value we use for `how` because both dataframes have 10 rows with matching years. [Here are some visual examples of how joins work.](https://docs.google.com/spreadsheets/d/1IeOdOlwJDz8ekPQbJWC5yt5rIYaRGbcX1nUmkdQ_3H8/edit#gid=1215158202)

3. The `on` argument tells pandas which column key we're going to match on. In this case, we want the years to match up.

4. The `validate` argument is optional, but I recommend you learn how to use it. The value we used, `'1:1'` means that 1 row in the left-hand dataframe will match up to exactly 1 row in the right-hand dataframe. The option `1:m` means that 1 row in the left-hand dataframe could match up to **many** rows in the right-hand dataframe. (Any time you use `m`, you're telling pandas that there _might_ be multiple matches.)


In [52]:
annual_summary = pd.merge(
    annual_cases,
    median_close_time,
    on='Year',
    how='outer',
    validate='1:1'
)

In [53]:
annual_summary

,Year,Case_Count,Close_Time_Days
0,2011,39708,86.215602
1,2012,46643,16.003565
2,2013,50309,7.965579
3,2014,53531,10.768397
4,2015,50334,4.334236
5,2016,50573,3.912419
6,2017,51325,3.639792
7,2018,57636,3.175874
8,2019,58633,3.705052
9,2020,56145,2.195741
